<a href="https://colab.research.google.com/github/yatta03/112-2PL/blob/main/med_daisho.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# set up chromedriver for selenium
!apt-get update
!apt install chromium-chromedriver
!cp /usr/lib/chromium-browser/chromedriver /usr/bin

!pip install selenium
!pip install pygsheets

Get:1 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Get:2 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Get:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Hit:4 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:5 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Hit:6 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Hit:7 https://ppa.launchpadcontent.net/c2d4u.team/c2d4u4.0+/ubuntu jammy InRelease
Get:8 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  Packages [891 kB]
Hit:9 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Get:10 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease [24.3 kB]
Hit:11 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:12 http://security.ubuntu.com/ubuntu jammy-security/main amd64 Packages [1,858 kB]
Get:13 http://security.ubuntu

In [ ]:
# import requests
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from bs4 import BeautifulSoup as bs
import json
import pandas as pd
import pygsheets
import re

In [ ]:
class Crawler:
  def __init__(self):
    chrome_options = webdriver.ChromeOptions()
    chrome_options.add_argument('--headless')
    chrome_options.add_argument("--disable-gpu")  # Required for Windows
    chrome_options.add_argument('--no-sandbox')
    chrome_options.headless = True
    self.driver = webdriver.Chrome(options=chrome_options)

  # wait until the button to be clickable, then click
  def waitClickBtn(self, byType, byName):
    try:
      element = WebDriverWait(self.driver, 5).until(EC.element_to_be_clickable((byType, byName)), "not found")
      element.click()
    except Exception as e:
      print(e)

  #  wait the locator or waittime to load the content
  def waitContent(self, waitTime, locator = None):
    if locator == None:
      self.driver.implicitly_wait(waitTime)
    else:
      WebDriverWait(self.driver, waitTime).until(EC.presence_of_element_located(locator), "not found")

  # wait until locator presents and return parsed html of given url
  def getSoup(self, url, locator = None):
    self.driver.get(url)
    if(locator):
      try:
          WebDriverWait(self.driver, 10).until(EC.presence_of_element_located(locator), "not found")
      except:
          print("locator not found")
    else:
      self.driver.implicitly_wait(5)
    htmlContent = self.driver.page_source

    soup = bs(htmlContent, 'html.parser')
    return soup

  # quit the driver
  def quit(self):
    self.driver.quit()

In [ ]:
daisho_url = "https://www.twtaisho.com.tw/p3_product_list.php?tid=2"

In [ ]:
# get page link of each medicine type
def get_med_type_link(mlink):
  crawler = Crawler()
  locator = (By.CSS_SELECTOR, ".footer-list.pdt")
  crawler.getSoup(mlink, locator)

  links = []
  sidelist = crawler.driver.find_elements(By.XPATH, "//ul[@class='footer-list pdt']/li/a")
  for sl in sidelist:
    # print(sl.get_attribute("href"))
    links.append(sl.get_attribute("href"))

  crawler.quit()
  return links

In [ ]:
# get link of medicine info
def get_med_link(plink):
  crawler = Crawler()
  locator = (By.CLASS_NAME, "pro-list-all")
  crawler.getSoup(plink, locator)

  links = crawler.driver.find_elements(By.XPATH, "//ul[@class='pro-list-all']/li/a")
  links = [l.get_attribute("href") for l in links]
  crawler.quit()
  return links

In [ ]:
def get_med(link):
  med = {}
  crawler = Crawler()
  locator = (By.CLASS_NAME, "p3detail_text")
  crawler.getSoup(link, locator)

  name = crawler.driver.find_element(By.XPATH, "//div[@class='chi_name']").text
  symptom = crawler.driver.find_element(By.XPATH, "//ul[@class='p3detail_text']/li/div[@class='b']").text
  img_path = crawler.driver.find_element(By.XPATH, "//div[@class='pic']/img").get_attribute("src")
  ingredients = crawler.driver.find_elements(By.XPATH, "//div[@class='p3_infor_data']/ul/li")
  ingredients = [i.text for i in ingredients]

  med["name"] = name
  med["img_path"] = img_path
  med["app_txt"] = ""
  med["symptom"] = symptom
  med["ingredient"] = ingredients
  med["link"] = link

  crawler.quit()
  return med

In [ ]:
med_type_list = get_med_type_link(daisho_url)
med_type_list = med_type_list[:-2]
med_type_list

['https://www.twtaisho.com.tw/p3_product_list.php?tid=2',
 'https://www.twtaisho.com.tw/p3_product_list.php?tid=18',
 'https://www.twtaisho.com.tw/p3_product_list.php?tid=19',
 'https://www.twtaisho.com.tw/p3_product_list.php?tid=3',
 'https://www.twtaisho.com.tw/p3_product_list.php?tid=6',
 'https://www.twtaisho.com.tw/p3_product_list.php?tid=29',
 'https://www.twtaisho.com.tw/p3_product_list.php?tid=20',
 'https://www.twtaisho.com.tw/p3_product_list.php?tid=30']

In [ ]:
links = []
medicines = []
for pl in med_type_list:
  links.extend(get_med_link(pl))

for l in links:
  medicines.append(get_med(l))

In [ ]:
daisho_med = pd.DataFrame(medicines)
print(daisho_med.shape)
daisho_med.head()

(16, 6)


,name,img_path,app_txt,symptom,ingredient,link
0,大正百保能感冒顆粒,https://www.twtaisho.com.tw/Upload/product/pro...,,緩解感冒之各種症狀(流鼻水、鼻塞、打噴嚏、咽喉痛、咳嗽、喀痰、發燒、頭痛、關節痛、肌肉痛),[Acetaminophen ..................................,https://www.twtaisho.com.tw/p3_product_detail....
1,大正百保能感冒錠,https://www.twtaisho.com.tw/Upload/product/pro...,,緩解感冒之各種症狀 (流鼻水、鼻塞、打噴嚏、咽喉痛、咳嗽、喀痰、畏寒、發燒、頭痛、關節痛、肌...,[],https://www.twtaisho.com.tw/p3_product_detail....
2,大正百保能感冒膠囊,https://www.twtaisho.com.tw/Upload/product/pro...,,緩解感冒之各種症狀(流鼻水、鼻塞、打噴嚏、咽喉痛、咳嗽、喀痰、發燒、頭痛、關節痛、肌肉痛),"[Acetaminophen.........................150mg, ...",https://www.twtaisho.com.tw/p3_product_detail....
3,大正百保能鼻炎膠囊,https://www.twtaisho.com.tw/Upload/product/pro...,,緩解過敏性鼻炎、枯草熱所引起之相關症狀(鼻塞、流鼻水、打噴嚏、眼睛及喉部搔癢),[Chlorpheniramine Maleate........................,https://www.twtaisho.com.tw/p3_product_detail....
4,大正百保能止咳糖漿,https://www.twtaisho.com.tw/Upload/product/pro...,,鎮咳、袪痰,[Codeine Phosphate ..............................,https://www.twtaisho.com.tw/p3_product_detail....


In [ ]:
gc = pygsheets.authorize(service_file='./pl-421203-a74b8641e6bc.json')
sheetUrl = "https://docs.google.com/spreadsheets/d/16_34--12pFGZ3alnL_mAuCEE5ne0aaxZ0sd8qlJzyOo/edit?usp=sharing"
sh = gc.open_by_url(sheetUrl)

In [ ]:
def dfToSheet(df, sh, title):
  try:
    ws = sh.worksheet_by_title(title)
    ws.set_dataframe(df, start=(0,0), nan='')
  except Exception as e:
    print(e)
def clearSheet(sh, title):
  try:
    ws = sh.worksheet_by_title(title)
    ws.clear()
  except Exception as e:
    print(e)

In [ ]:
clearSheet(sh, "daisho")
dfToSheet(daisho_med, sh, "daisho")